<font color=gray>Oracle Cloud Infrastructure Data Science Sample Notebook

Copyright (c) 2021 Oracle, Inc.  All rights reserved. <br>
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

# Uploading Larger Size Model Artifact Using OCI Python SDK 

This notebook demonstrates simple solution for OCI Python SDK which allows data scientists to upload larger model artifacts and eliminate the timeout error that is experienced by data scientists when the artifact is large. It shows end-to-end steps from setting up the configuration till uploading the model artifact.

## Pre-requisites to Running this Notebook 

* We recommend that you run this notebook in an OCI Data Science Notebook Session. Use a conda environment that has a `oci>=2.43.2`. You can always upgrade oci by executing `!pip install oci --upgrade` in your notebook cell. 
* You need access to the public internet

In [ ]:
import os
import logging
import json

import oci
from oci.data_science import DataScienceClient
from oci.data_science.models import CreateModelDetails, CreateProjectDetails, \
    CreateModelProvenanceDetails, Project, Model, UpdateModelDetails

data_science_models = oci.data_science.models

In [ ]:
print(oci.__version__)

In [ ]:
# here we assume that you are authenticating with OCI resources using user principals (config + key)
# set resource_principal == True if you are using resource principal instead of user principal 
resource_principal = False

if not resource_principal: 
    config = oci.config.from_file()
    data_science = DataScienceClient(config)
else: 
    config = {}
    auth = oci.auth.signers.get_resource_principals_signer()
    data_science = DataScienceClient(config, signer=auth)

In [ ]:
# Creating the metadata about the model: 
create_model_details_object = data_science_models.CreateModelDetails()
create_model_details_object.compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']
create_model_details_object.display_name = f"<replace-with-your-object-display-name>"
create_model_details_object.project_id = os.environ['PROJECT_OCID']

In [ ]:
# create the model object: 
model = data_science.create_model(create_model_details_object)

In [ ]:
# print the model OCID: 
model_id = json.loads(str(model.data))['id']
print(model_id)

In [ ]:
# next we download a sample boilerplate model artifact and setting the model artifact path. 
!wget https://github.com/oracle/oci-data-science-ai-samples/blob/master/model_catalog_examples/artifact_boilerplate/artifact_boilerplate.zip 
artifact_path = 'artifact_boilerplate.zip'

In [ ]:
with open(artifact_path,'rb') as artifact_file:
    data_science.create_model_artifact(model_id,
                                       artifact_file,
                                       content_disposition='attachment; filename="{artifact_path}"')

In [ ]:
# Go to the OCI Console and confirm that your model was successfully created. 